In [8]:
!pip install transformers --quiet

import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [9]:
test_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/test.csv")
print("testShape:", test_df.shape)
test_df.head()


testShape: (1707, 2)


,id,text
0,0,she wanted to fight over every single little t...
1,1,"anyway, back to tuesday."
2,2,she shrieked at the dog to go back.
3,3,yelling for everyone to get back or get inside...
4,4,still kind of freaky.


In [10]:
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class TestDataset(Dataset):
    def __init__(self, df, max_len=256):
        self.texts = df["text"].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        enc = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze()
        }

test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
class RobertaMultiLabel(nn.Module):
    def __init__(self, model_name, num_labels=5, dropout=0.3):
        super().__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]   # CLS token
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)
        return logits

model = RobertaMultiLabel(MODEL_NAME, num_labels=5).to(device)

weight_path = "/kaggle/input/roberta-emotion-best/roberta_emotion_best/pytorch_model.bin"

state_dict = torch.load(weight_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

print("Model load successfully!")


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model load successfully!


In [12]:
labels = ["anger", "fear", "joy", "sadness", "surprise"]

all_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_preds.append(probs)

all_preds = np.vstack(all_preds)

THRESHOLD = 0.5
binary_preds = (all_preds >= THRESHOLD).astype(int)

binary_preds[:3]


Predicting:   0%|          | 0/107 [00:00<?, ?it/s]

array([[1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0]])

In [13]:
submission = pd.DataFrame(binary_preds, columns=labels)
submission.insert(0, "id", test_df["id"])

submission.to_csv("submission.csv", index=False)
submission.head()
# print("submission.csv saved!")


,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,1,0
1,1,0,0,0,0,0
2,2,1,1,0,0,0
3,3,0,1,0,0,0
4,4,0,1,0,0,1


In [11]:
submission.head()

,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,1,0
1,1,0,0,0,0,0
2,2,1,1,0,0,0
3,3,0,1,0,0,0
4,4,0,1,0,0,1
